# Deep Learning Toolkit for Splunk - Barebone Notebook

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [6]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader,TensorDataset,Dataset
from torch.autograd import Variable as V
import time
from sklearn.metrics import classification_report, confusion_matrix,roc_curve, auc
from sklearn.model_selection import train_test_split
import string
import torch.nn as nn
import collections
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.pipeline import Pipeline

import torch.optim as optim
MODEL_DIRECTORY = "/srv/app/model/data/detect_dns_data_exfiltration_using_pretrained_model_in_dsdl"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone epochs=10 batch_size=1 s from feature_* into app:barebone_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("barebone_model")
print(df.describe())
print(param)

## Stage 2 - create and initialize a model

In [7]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently

class DNSExfiltration(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.layer_1 = nn.Linear(input_size, 256) 
        self.layer_2 = nn.Linear(256, 256)
        self.layer_out = nn.Linear(256, 1) 
        
        
        self.relu = nn.ReLU()
        self.sigmoid =  nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.5)
        

    def forward(self, input):
        x = self.relu(self.layer_1(input))
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.layer_out(x))
        return x
    

keys = list(string.printable.strip())
print (len(list(string.printable.strip())))
    
def init(df,param):
    model = DNSExfiltration(102)
    model.load_state_dict(torch.load(MODEL_DIRECTORY+'/detect_dns_data_exfiltration_using_pretrained_model_in_dsdl.pt',map_location=torch.device('cpu')))
    model = model.to(device)
    model.eval()
    return model

94


In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [9]:
# apply your model
# returns the calculated results
text_rows=[]

def index_chars(x):
    request_chars = {}
    for i in range(len(x)):
        request_chars[keys.index(x[i])] = request_chars.get(keys.index(x[i]),0)+1
    text_rows.append(request_chars)
    
def sub_domain_count(x):
    
        
def predict(dataloader,model):
        predict_label = []
        for batch in (dataloader):
            
            #convert to 1d tensor
            predictions = model(batch.to(device))
            output  = (predictions >= 0.5).float() 
            predict_label.extend(output)
        predict_label = [x.cpu().detach().numpy().item() for x in predict_label]
        return output,predict_label



def apply(model,df,param):
    df['request'].apply(lambda x: index_chars(x))
    text = pd.DataFrame(text_rows,columns=list(range(0, 94)))
    text.reset_index(drop=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    text.fillna(0,inplace =True)
    df = pd.concat([text,df],axis=1)
    df = torch.FloatTensor(df.values)
    # request without tld
    df ['request_without_tld'] = [e.replace(k, '') for e, k in zip(df['request'].astype('str'), df[''].astype('str'))]
    # length of domain
    df["len"] = df["request"].apply(len)
    
    # number of subdomains
    df["subdomains_count"] = df["request"].apply(lambda x: len(str(df["request"]).split('.')))
    
    #
    
    dataloader = DataLoader(df, shuffle=True, batch_size=256)
    predict_is_exfiltration_proba, predict_is_exfiltration = predict(train_dataloader)
    output = pd.DataFrame()
    output['pred_is_dns_data_exfiltration_proba'] = predict_is_exfiltration_proba
    output['pred_is_dns_data_exfiltration'] = predict_is_exfiltration
    return output

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [10]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = DNSExfiltration(102)
    model.load_state_dict(torch.load(MODEL_DIRECTORY+'/detect_dns_data_exfiltration_using_pretrained_model_in_dsdl.pt',map_location=torch.device('cpu')))
    model = model.to(device)
    model.eval()
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code